# Final CHO Model
This notebook is to asses the validity of our reconstruction and how complete it is.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import validate_sbml_model, write_sbml_model
from tqdm.notebook import tqdm
from google_sheet import GoogleSheet

### 1. Retrieve information from the Google Sheet datasets reactions and metabolites

In [3]:
##### ----- Generate datasets from Google Sheet ----- #####

#Credential file
KEY_FILE_PATH = 'credentials.json'

# #CHO Network Reconstruction + Recon3D_v2 Google Sheet ID
# SPREADSHEET_ID = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

#CHO Network Reconstruction + Recon3D_v3 Google Sheet ID
SPREADSHEET_ID = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_met = 'Metabolites'
sheet_rxns = 'Rxns'
shee_attributes = 'Attributes'

metabolites = sheet.read_google_sheet(sheet_met)
rxns = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(shee_attributes)

### 2. Build a model and feed it the information from the df generated

In [3]:
##### ----- Create a model and add reactions ----- #####
model = Model("iCHO")
lr = []
for _, row in rxns.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)    
model.add_reactions(lr)
model

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


Name,iCHO
Memory address,1582dba60
Number of metabolites,0
Number of reactions,11375
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [ ]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(tqdm(model.reactions)):
    print(r.id)
    r.build_reaction_from_string(rxns['Reaction Formula'][i])
    r.name = rxns['Reaction Name'][i]
    r.subsystem = rxns['Subsystem'][i]
    r.gene_reaction_rule = str(rxns['GPR_final'][i])
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])    

In [5]:
##### ----- Add information for each metabolite ----- #####
metabolites_dict = metabolites.set_index('BiGG ID').to_dict('dict')
metabolites_dict['Name']
for met in model.metabolites:
    met.name = metabolites_dict['Name'][f'{met}']
    met.formula = metabolites_dict['Formula'][f'{met}']
    met.compartment = metabolites_dict['Compartment'][f'{met}'].split(' - ')[0]    
    

In [7]:
##### ----- Build the S matrix ----- #####
S = cobra.util.create_stoichiometric_matrix(model, array_type='dense')
model.S = S

In [ ]:
##### ----- Save the model ----- #####
write_sbml_model(model, "iCHOv3_CHO+Recon_v3.xml")

In [ ]:
##### ----- Test for errors in the recostruction ----- ######

# import tempfile
# from pprint import pprint
# from cobra.io import write_sbml_model, validate_sbml_model
# with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
#     write_sbml_model(model, filename=f_sbml.name)
#     report = validate_sbml_model(filename=f_sbml.name)
# pprint(report)

from cobra.io import read_sbml_model, validate_sbml_model
(_, errors) = validate_sbml_model("iCHOv3_CHO+Recon_v3.xml")
errors

### Blocked reactions and Dead-Ends

In [4]:
##### ----- Read Model ----- #####
from cobra.io import read_sbml_model
model = read_sbml_model("iCHOv3_CHO+Recon_v3.xml")

for rxn_exchange in model.exchanges:
    rxn_exchange.bounds = (-1000, 1000)

No objective coefficients in model. Unclear what should be optimized


In [ ]:
##### ----- FVA ----- #####
import importlib
# from utils import runMinMax_GF
import utils
importlib.reload(utils)
model.solver = 'gurobi'


for rxn in model.reactions:
    rxn.bounds = -1000, 1000
minmax = utils.runMinMax_Single(model, end_rxn_index=None)

##### Print Reactions with the min and max fluxes #####
with open('FVA_Results.txt', 'w') as f:
    for i, j in enumerate(minmax):
        print(model.reactions[i].id, "Min: ",j[0], "Max: ", j[1], file=f)
        
##### Print Blocked Reactions #####
with open('Blocked_Reactions.txt', 'w') as f:
    for i, j in enumerate(minmax):
        if j[0] == 0 and j[1] == 0:
            print(model.reactions[i].id, "Min: ",j[0], "Max: ", j[1], file=f)
            

In [ ]:
blocked_reactions = []
for i, j in enumerate(minmax):
        if j[0] == 0 and j[1] == 0:
            blocked_reactions.append(model.reactions[i].id)
            
print('##### ----- Blocked Reactions ----- #####')
print(f'The model has {len(model.reactions)} total reactions')
print(f'The model has {len(blocked_reactions)} ({round(len(blocked_reactions)/len(model.reactions)*100)}%) blocked reactions')

In [ ]:
import cobra

def find_dead_end_metabolites(model):
    dead_ends = []
    
    exchanges = list(model.exchanges)
    
    for metabolite in model.metabolites:
        
        reactions = [rxn for rxn in metabolite.reactions]
        
        if any(item in reactions for item in exchanges):
            continue
        else:
            producers = [rxn for rxn in reactions if rxn.get_coefficient(metabolite) < 0]
            consumers = [rxn for rxn in reactions if rxn.get_coefficient(metabolite) > 0]

            if len(producers) == 0 or len(consumers) == 0:
                dead_ends.append(metabolite)
    
    return dead_ends


# Find dead-end metabolites
dead_end_metabolites = find_dead_end_metabolites(model)

# Print dead-end metabolites
print("Dead-end metabolites:")
for metabolite in dead_end_metabolites:
    print(metabolite.id)

len(dead_end_metabolites)

In [ ]:
from cobra.flux_analysis import gapfill

universal_model = cobra.io.read_sbml_model('universal_model.xml')
solution = gapfill(model, universal_model, demand_reactions=False)

In [5]:
##### ----- Detect Dead-Ends ----- #####
from utils import detect_dead_ends
dead_ends = detect_dead_ends(model)

with open('Dead_ends.txt', 'w') as f:
    for i,j in enumerate(dead_ends):
        if j:
            print(model.metabolites[i], file=f)

for i,j in enumerate(dead_ends):
    if j:
        print("\n", model.metabolites[i],": ")
        for met_rxn in model.metabolites[i].reactions:
            print(met_rxn)

In [15]:
with open('Dead_ends.txt', 'r') as f:
    for line in f:
        if line.strip().endswith('_e'):
            rxn = line.strip().replace('EX_', '')
            # Automated Attributes print
            print(f'Than\t{line.strip()}\t{rxn} -->\tEXCHANGE/DEMAND REACTION\t\t\t\t\t\t1\t0\t1000\t0')
            # Automated Rxns print
            print(f'Than\t{line.strip()}\tExchange of {rxn}\t {rxn} -->\tEXCHANGE/DEMAND REACTION')

Than	11docrtsl_e	Exchange of 11docrtsl_e	 11docrtsl_e -->	EXCHANGE/DEMAND REACTION
Than	11docrtstrn_e	Exchange of 11docrtstrn_e	 11docrtstrn_e -->	EXCHANGE/DEMAND REACTION
Than	13dampp_e	Exchange of 13dampp_e	 13dampp_e -->	EXCHANGE/DEMAND REACTION
Than	17ahprgnlone_e	Exchange of 17ahprgnlone_e	 17ahprgnlone_e -->	EXCHANGE/DEMAND REACTION
Than	1a25dhvitd3_e	Exchange of 1a25dhvitd3_e	 1a25dhvitd3_e -->	EXCHANGE/DEMAND REACTION
Than	2hydog_e	Exchange of 2hydog_e	 2hydog_e -->	EXCHANGE/DEMAND REACTION
Than	2obut_e	Exchange of 2obut_e	 2obut_e -->	EXCHANGE/DEMAND REACTION
Than	7klitchol_e	Exchange of 7klitchol_e	 7klitchol_e -->	EXCHANGE/DEMAND REACTION
Than	acrn_e	Exchange of acrn_e	 acrn_e -->	EXCHANGE/DEMAND REACTION
Than	ahandrostan_e	Exchange of ahandrostan_e	 ahandrostan_e -->	EXCHANGE/DEMAND REACTION
Than	alaarggly_e	Exchange of alaarggly_e	 alaarggly_e -->	EXCHANGE/DEMAND REACTION
Than	alaasnleu_e	Exchange of alaasnleu_e	 alaasnleu_e -->	EXCHANGE/DEMAND REACTION
Than	alaglylys_e	Ex

In [ ]:
##### ----- Create demand reactions ----- #####
# with open('Dead_ends.txt', 'r') as f:
#     for dead_meat in f:
#         print(model.metabolites.get_by_id(dead_meat.rstrip()))
#         model.add_boundary(model.metabolites.get_by_id(dead_meat.rstrip()), type="sink")

### Test CHO - Recon GEM

In [ ]:
# iCHO_recon3dfrom cobra.io import read_sbml_model
# read_sbml_model(".xml")

model_EX = [i for i, rxn in enumerate(model.reactions) if 'EX_' in rxn.id]
model_SK = [i for i, rxn in enumerate(model.reactions) if 'SK_' in rxn.id]
model_DM = [i for i, rxn in enumerate(model.reactions) if 'DM_' in rxn.id]
for i in model_EX:
    model.reactions[i].bounds = -1000, 1000

for i in model_SK:
    model.reactions[i].lower_bound = -1000

for i in model_DM:
    model.reactions[i].bounds = 0, 1000
    

In [ ]:
model.objective = "biomass"
sol1 = model.optimize()
print(sol1.objective_value)

model.objective = "biomass_prod"
sol2 = model.optimize()
print(sol2.objective_value)

model.objective = "biomass_producing"
sol3 = model.optimize()
print(sol3.objective_value)


In [ ]:
##### ----- Test model KOs ----- #####
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))